# <span style="color:red"> Introduction to Python, Part 3 </span>

In [63]:
!pip install datascience

In [64]:
# Import code libraries or "modules" in Python lingo

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import importlib
import sys
from datascience import *
sns.set_style("whitegrid")
%matplotlib inline

### <span style="color:blue"> Tables </span>

For illustration let's make simple table

In [65]:
flowers = Table().with_columns(
    "name", make_array("lotus", "sunflower", "rose"),
    "petals", make_array(8, 34, 5),
    "color", make_array("pink", "yellow", "red")
)
flowers

name,petals,color
lotus,8,pink
sunflower,34,yellow
rose,5,red


Rows of the table represent objects (in his case, flowers). 
Columns of the table represent features of objects (in this case, name, number of petals, color). This data format is very common; many data sets come in this format or can be converted to this format.

The datascience module has many functions for operating on tables. 

Here is a 
[cheatsheet](https://github.com/wstuetzle/STAT180/blob/master/Computing/data8_sp17_midterm_ref_sheet.pdf).

Here is a [tutorial](http://data8.org/datascience/tutorial.html)

Let's try some of the functions / methods

In [66]:
print(flowers.num_rows)       # number of rows
print(flowers.num_columns)    # number of columns

3
3


In [67]:
flowers.labels     # names of columns

('name', 'petals', 'color')

In [68]:
flowers.column("name")     # extract a column

array(['lotus', 'sunflower', 'rose'],
      dtype='<U9')

In [69]:
flowers.select("name", 1)    

# makes a new table with only the selected columns
# columns can be identified by name or index
# remember: indexing start at 0                    

name,petals
lotus,8
sunflower,34
rose,5


In [70]:
# The table  flowers  is not changed

flowers

name,petals,color
lotus,8,pink
sunflower,34,yellow
rose,5,red


In [71]:
# But we can define new table

flower_names_only = flowers.select("name", 1) 
flower_names_only

name,petals
lotus,8
sunflower,34
rose,5


In [72]:
# Make a new table that has only the first two rows

flowers.take(make_array(0, 1))    



name,petals,color
lotus,8,pink
sunflower,34,yellow


In [73]:
# Sort the column "petals" and shuffels the other columns along

sorted_flowers = flowers.sort("petals")
sorted_flowers

name,petals,color
rose,5,red
lotus,8,pink
sunflower,34,yellow


In [74]:
flowers

# again, flowers did not change - good!

name,petals,color
lotus,8,pink
sunflower,34,yellow
rose,5,red


In [75]:
# Select rows by values of a column

flowers.where("petals", are.above(5))    

name,petals,color
lotus,8,pink
sunflower,34,yellow


### <span style="color:blue"> Who survived when the Titanic sank? (Part 1) </span>

More info on the sinking of the Titanic in [Encyclopedia Titanica](http://www.encyclopedia-titanica.org) or Wikipedia

A "comma separated values" (csv) version of the dataset we will be analyzing is on Canvas. You can downoad it and look at in using Excel. Let's do that. 

The spreadsheet has 1,310 rows, but the first one is the column labels. So there are data on 1,309 passengers.

A description of the dataset is on Canvas.

**Here are the column labels and their meanings**
```
    pclass     cabin class
    survived   0 = "no", 1 = "yes"
    name
    sex
    age
    sibsp      number of siblings or spouses aboard
    parch      number of parents/children aboard
    ticket     ticket number
    fare       passenger fare (in British pounds)
    cabin      cabin number
    embarked   port of embarkation (C + cherbourg; Q = Queenstown; S = Southampton
    boat       lifeboat number
    body       body identification number
    home.dest  home/destination 
```

**Note**

If you want to run this notebook on Azure, its easiest to upload the notebook, the file "datascience_extensions.py" (on Canvas) and the file "titanic3.csv" (on Canvas) and then replace the next two cells (below) by a single code cell
with the following two lines:

from datascience_extensions import * <br>
titanic = Table.read_table("titanic3.csv")

In [76]:
# Import some new methods for class Table
# Add IDS directory to search path
course_dir = "/Users/wxs/Dropbox/IDS/Git-reps/STAT180/"
computing_dir = course_dir + "Computing"

if computing_dir not in sys.path:
    sys.path.append(computing_dir)

from datascience_extensions import *

# Reload the extensions after we make a change
# Importing it again does not work - a module is imported only once
module_name = "datascience_extensions"
importlib.reload(sys.modules[module_name])

<module 'datascience_extensions' from '/Users/wxs/Dropbox/IDS/Git-reps/STAT180/Lectures/datascience_extensions.py'>

In [77]:
# Read the data into a table "titanic"
data_dir = course_dir + "Data/"
titanic_filename = "titanic3.csv"
titanic_pathname = data_dir + titanic_filename
titanic = Table.read_table(titanic_pathname)

In [78]:
# Display the first three rows

titanic.take(make_array(0, 1, 2))

pclass,survived,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,body,home.dest
1,1,"Allen, Miss. Elisabeth Walton",female,29,0,0,24160,211.338,B5,S,2,nan,"St Louis, MO"
1,1,"Allison, Master. Hudson Trevor",male,0.9167,1,2,113781,151.55,C22 C26,S,11,nan,"Montreal, PQ / Chesterville, ON"
1,0,"Allison, Miss. Helen Loraine",female,2,1,2,113781,151.55,C22 C26,S,nan,nan,"Montreal, PQ / Chesterville, ON"


Rows of the table correspond to passengers <br> 
Columns correspond to properties (or "features") of passengers

We often refer to the rows as "cases" and to the columns as "variables". Unfortunate: the term "variable" is overloaded.

**Some entries in the table are "nan", indicating that the value is missing**

In [79]:
# Let's see how big the table is

titanic.shape()

# shape is a method defined in the module  datascience_extensions

[1310, 14]

In [80]:
# So the table "titanic" has 1,310 rows, numbered 0..1309
# The spreadsheet "titanic3.csv" has also 1,310 rows, but row 1 contains 
# the column labels. So we only have data on 1,309 passengers.
# Something is amiss. Let's look at the last ow of the table

titanic.take(1309)

pclass,survived,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,body,home.dest
nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan


In [81]:
# "nan" is a code for "missing"
# So Table.read_table has read an empty row (all elements missing). 
# Let's get rid of that row

titanic = titanic.take(np.arange(1309))
titanic.take(1308)

# Now the last row looks legit

pclass,survived,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,body,home.dest
3,0,"Zimmerman, Mr. Leo",male,29,0,0,315082,7.875,nan,S,nan,nan,nan


In [82]:
# Complication:
# There are missing values (nan) in the table. 
# 
# Let's count the missing values for each variable
nan_count = titanic.count_nan()
nan_count

# count_nan is a method in the module datascience_extensions

{'age': 263,
 'boat': 823,
 'body': 1188,
 'cabin': 1014,
 'embarked': 2,
 'fare': 1,
 'home.dest': 564,
 'name': 0,
 'parch': 0,
 'pclass': 0,
 'sex': 0,
 'sibsp': 0,
 'survived': 0,
 'ticket': 0}

### <span style="color:blue"> Dealing with missing values </span>

Missing values are a common problem in many "real world" data sets

There is no simple recipe.

One option is "imputation": guess the missing values for a row (passenger), possibly based on the values of the features that are non-missing (present).

* Guess a missing age by the average of all present ages
* Guess a missing age by the average age of passengers with the same sex, pclass, and with present age



For some questions the fact that the dataset has missing values does not matter. For example we can calculate the percentage of survivors because the value of "survived" is present for all passengers.

If a question involves features with missing values we can either throw up our hands or qualify our answer as in
"for passengers for whom we know the age, higher age goes with higher (or lower?) survival rate"

### <span style="color:blue"> Computing some simple summaries </span>

In [83]:
# Survival rate - what fraction of passengers survived?

np.mean(titanic.column("survived"))

0.3819709702062643

In [84]:
# To get the percentage of survivors

round(100 * np.mean(titanic.column("survived")))

38.0

In [85]:
# What's the average age?

np.mean(titanic.column("age"))

# Whoops - age is missing for 263 passengers, so we can't compute the average age

nan

In [86]:
# Let's compute the average age for passengers with age present

round(np.mean(titanic.select("age").take_complete_rows().column("age")), 0)

# Let's parse this statement

30.0

In [87]:
# Let's see if there is a difference in the average age between males and females
# and between passengers in different cabin classes

titanic_sap = titanic.select("sex", "pclass", "age").take_complete_rows()
titanic_sap.shape()

[1046, 3]

In [88]:
round(np.mean(titanic_sap.where("sex", are.equal_to("male")).column("age")))

31.0

In [89]:
round(np.mean(titanic_sap.where("sex", are.equal_to("female")).column("age")))

# male passengers on average are two years older than female passengers

29.0

In [90]:
# Let's look at age versus cabin class

round(np.mean(titanic_sap.where("pclass", are.equal_to(1)).column("age")))

39.0

In [91]:
round(np.mean(titanic_sap.where("pclass", are.equal_to(3)).column("age")))

# Passengers in 1st class are on avergage much older than passengers in 
# 3rd class. Conjecture: passengers in 3rd class are emigrants who
# tend to be young. There are other indications in the data supporting this
# conjecture

25.0

In [92]:
# Let's save the titanic table for future use

titanic_table_pathname = data_dir + "titanic3-table.csv"
titanic.to_csv("titanic3_table_pathname")